In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# change path - might have to add shortcut to drive
# %cd drive/MyDrive/dcp/
# %ls
%cd /content/drive/MyDrive/CS8803_DLM_Project_Shared/dcp/
%ls

/content/drive/.shortcut-targets-by-id/19Y1dy9JseYCYXPQnwNARoryGvoo7e12a/CS8803_DLM_Project_Shared/dcp
checkpoints/  main.ipynb   model_net_checkpoints/  __pycache__/
data/         main.py      model.py                readme.md
data.py       main_sim.py  pretrained/             util.py


In [ ]:
!pip install tensorboardX
!pip install scipy==1.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:



from __future__ import print_function
import os
import gc
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data import ModelNet40
from model import DCP
from util import transform_point_cloud, npmat2euler
import numpy as np
import numpy.random as rand
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from tqdm import tqdm
print('Imports successful!')

Imports successful!


In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Load our simpified dataset for DCP
class LidarDataset(Dataset):
    def __init__(self, pairs_file, root):
        self.pairs = []
        with open(pairs_file, 'r') as f:
            for line in f:
                fields = line.strip().split()
                # get folder name of the point cloud data (e.g. expXX)
                folder_name = fields[0]
                # get lidar path
                lidar1 = os.path.join(root, folder_name, "cloud", fields[1])
                lidar2 = os.path.join(root, folder_name, "cloud", fields[2])
                # get transformation
                transformation = [float(x) for x in fields[5:]]
                self.pairs.append((lidar1, lidar2, transformation))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        # load into numpy arrays
        lidar1 = np.load(self.pairs[index][0])
        lidar2 = np.load(self.pairs[index][1])
        trans = np.array(self.pairs[index][2], dtype=np.float32).reshape(4, 4)
        
        # augment
        lidar1, lidar2, trans = apply_augmentation(lidar1, lidar2, trans)
        
        # convert to tensor and organize into (C, H, W)
        lidar1 = torch.tensor(np.transpose(lidar1, (2, 0, 1)))
        lidar2 = torch.tensor(np.transpose(lidar2, (2, 0, 1)))
        trans = torch.tensor(trans)
        return lidar1, lidar2, trans


root = "../dataset"

batch_size = 128
num_workers = 12
train_pairs_file = "../sim_dataset/train.txt"
train_dataset = LidarDataset(train_pairs_file, root)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

test_pairs_file = "../sim_dataset/test.txt"
test_dataset = LidarDataset(test_pairs_file, root)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

val_pairs_file = "../sim_dataset/validation.txt"
val_dataset = LidarDataset(val_pairs_file, root)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

print("Number of samples in train set:", len(train_dataset))
print("Number of samples in test set:", len(test_dataset))
print("Number of samples in validation set:", len(val_dataset))

Number of samples in train set: 68163
Number of samples in test set: 27014
Number of samples in validation set: 9788


In [ ]:
# Training on ModelNet40
!python main.py --exp_name=dcp_v1 --model=dcp --emb_nn=dgcnn --pointer=identity --head=svd

Streaming output truncated to the last 5000 lines.

        [[ 0.8970, -0.3290,  0.2952],
         [ 0.3752,  0.9198, -0.1148],
         [-0.2338,  0.2137,  0.9485]],

        [[ 0.7952, -0.4758,  0.3759],
         [ 0.5380,  0.8396, -0.0756],
         [-0.2796,  0.2623,  0.9236]],

        [[ 0.6487, -0.4056,  0.6439],
         [ 0.7219,  0.5959, -0.3519],
         [-0.2410,  0.6931,  0.6794]],

        [[ 0.8280, -0.3817,  0.4108],
         [ 0.4699,  0.8720, -0.1368],
         [-0.3060,  0.3063,  0.9014]],

        [[ 0.6698, -0.6076,  0.4269],
         [ 0.7300,  0.6441, -0.2285],
         [-0.1361,  0.4647,  0.8749]],

        [[ 0.9803, -0.0754,  0.1823],
         [ 0.1315,  0.9388, -0.3183],
         [-0.1472,  0.3360,  0.9303]],

        [[ 0.8648, -0.4725,  0.1701],
         [ 0.4941,  0.8610, -0.1204],
         [-0.0895,  0.1881,  0.9781]],

        [[ 0.9514, -0.0259,  0.3067],
         [ 0.0930,  0.9741, -0.2062],
         [-0.2934,  0.2247,  0.9292]],

        [[ 0.8703, -

In [ ]:
# Testing on ModelNet40
!python main.py --exp_name=dcp_v1 --model=dcp --emb_nn=dgcnn --pointer=identity --head=svd --eval

In [ ]:
# Training on sim dataset
!python main_sim.py --exp_name=dcp_v1 --model=dcp --emb_nn=dgcnn --pointer=identity --head=svd --eval --dataset='sim'

/content/drive/.shortcut-targets-by-id/19Y1dy9JseYCYXPQnwNARoryGvoo7e12a/CS8803_DLM_Project_Shared/dcp/main_sim.py:688: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if args.model_path is '':
Namespace(exp_name='dcp_v1', model='dcp', emb_nn='dgcnn', pointer='identity', head='svd', emb_dims=512, n_blocks=1, n_heads=4, ff_dims=1024, dropout=0.0, batch_size=32, test_batch_size=10, epochs=250, use_sgd=False, lr=0.001, momentum=0.9, no_cuda=False, seed=1234, eval=True, cycle=False, gaussian_noise=False, unseen=False, num_points=1024, dataset='sim', factor=4, model_path='')
  0% 0/212 [00:00<?, ?it/s]testestss:   0% 0/212 [00:00<?, ?it/s]
  0% 0/212 [00:00<?, ?it/s]
  0% 0/212 [00:05<?, ?it/s]
Traceback (most recent call last):
  File "/content/drive/.shortcut-targets-by-id/19Y1dy9JseYCYXPQnwNARoryGvoo7e12a/CS8803_DLM_Project_Shared/dcp/main_sim.py", line 713, in <module>
    main()
  File "/content/drive/.shortcut-targets-by-id/19Y1dy9JseYCYXPQnwNARoryGvoo7e12a/CS8803_DLM_Project

In [ ]:
with open("../sim_dataset/new_test.txt") as f:
  print(f.readline())

sim4 0.npy 8.npy 2.59423249171332e-05 0.09852500897851638 0.9999999999409875 7.274048083809527e-06 -8.069250882336851e-06 5.754487134856934e-06 -7.274013352358079e-06 0.9999999999642811 4.304193556679727e-06 -2.186805839009677e-05 8.069282190973444e-06 -4.304134860585983e-06 0.9999999999581802 -1.2715271218821655e-05 0.0 0.0 0.0 1.0

